- I condider Repeated First Price Auction(hereinafter called repeated FPA), not second price auction.
- In addition, I also consider battle of sexes for supplementary.

- Full Information: Each player knows their own value (v) which is fixed across all rounds
- Full Feedback: After each round, each player observes all bids from all players, not just their own outcome

This setting allows players to use opponent's bid history directly in their learning algorithms.

# Project 3

# Part 1

## repeated FPA

### Algorithms and Parameters

1. 1_empirical: Empirical algorithm - always maximizes current round expected utility based on past opponent's bid data. 
2. 2_ew: EW - Exponential Weight with learning_rate
3. 3_FTL: Follow The Leader algorithm - selects arm with highest cumulative payoff
4. 4_uniform_guessing: Uniform Guessing algorithm - randomly selects bids uniformly
5. 5_exploitation: Exploitation algorithm - waits and exploits when opponent bids low

- n_rounds: 1000 - number of rounds per simulation
- k: 100 - number of discrete arms (discretization level)
- n_mc: 100 - number of Monte Carlo simulation runs
- h: scaling parameter (default: value) - used in Exponential Weight algorithms
- value (v): 10.0 - player's value for the item (default)
- learning_rate: sqrt(log(k) / n) - learning rate for Exponential Weight algorithms (default for flexible)
  Optimal learning rate: epsilon = sqrt(log(k) / T)
- observation_rounds: 5 - number of observation rounds for exploitation algorithm (default)

In [6]:
import numpy as np
import sys, importlib
from pathlib import Path
sys.path.insert(0, str(Path('algorithm').resolve()))

empirical, ew, exploitation = [importlib.import_module(m) for m in ['1_Empirical', '2_ew', '5_exploitation']]
# Import new algorithms: FTL and Uniform Guessing
ftl = importlib.import_module('3_FTL')
uniform_guessing = importlib.import_module('4_uniform_guessing')

import repeated_FPA
from repeated_FPA import run_repeated_fpa, plot_results

In [12]:
# Analyze convergence pattern: Does each MC run converge to one equilibrium, 
# or does each MC run randomly select between the two equilibria?

import numpy as np

# Analyze each MC run's final convergence pattern
n_mc = len(results_flexible_vs_flexible['actions1'])
n_rounds = len(results_flexible_vs_flexible['actions1'][0])
last_n_rounds = 1000  # Analyze last 1000 rounds

# For each MC run, check what action profiles appear in the last 1000 rounds
mc_convergence_pattern = []

for mc_run in range(n_mc):
    actions1 = results_flexible_vs_flexible['actions1'][mc_run]
    actions2 = results_flexible_vs_flexible['actions2'][mc_run]
    
    # Get last 1000 rounds
    final_actions1 = actions1[-last_n_rounds:]
    final_actions2 = actions2[-last_n_rounds:]
    
    # Count action profiles
    profile_counts = np.zeros(4)  # (A,A), (A,B), (B,A), (B,B)
    for i in range(last_n_rounds):
        a1 = final_actions1[i]
        a2 = final_actions2[i]
        profile_idx = a1 * 2 + a2
        profile_counts[profile_idx] += 1
    
    # Normalize to frequencies
    profile_freq = profile_counts / last_n_rounds
    
    # Determine convergence pattern
    # If one profile has frequency > 0.95, it's "converged to that equilibrium"
    # If both (A,A) and (B,B) have significant frequency (> 0.05), it's "mixed within MC run"
    if profile_freq[0] > 0.95:  # (A, A)
        pattern = "converged_to_AA"
    elif profile_freq[3] > 0.95:  # (B, B)
        pattern = "converged_to_BB"
    elif profile_freq[0] > 0.05 and profile_freq[3] > 0.05:
        pattern = "mixed_within_run"
    else:
        pattern = "other"
    
    mc_convergence_pattern.append({
        'pattern': pattern,
        'freq_AA': profile_freq[0],
        'freq_AB': profile_freq[1],
        'freq_BA': profile_freq[2],
        'freq_BB': profile_freq[3]
    })

# Count patterns
pattern_counts = {}
for mc_data in mc_convergence_pattern:
    pattern = mc_data['pattern']
    pattern_counts[pattern] = pattern_counts.get(pattern, 0) + 1

print("=== MC Run Convergence Pattern Analysis ===")
print(f"Total MC runs: {n_mc}")
print(f"\nConvergence patterns in last {last_n_rounds} rounds:")
for pattern, count in sorted(pattern_counts.items()):
    percentage = count / n_mc * 100
    print(f"  {pattern}: {count} runs ({percentage:.1f}%)")

# Show detailed statistics
print(f"\n=== Detailed Statistics ===")
print(f"Runs that converged to (A, A): {pattern_counts.get('converged_to_AA', 0)}")
print(f"Runs that converged to (B, B): {pattern_counts.get('converged_to_BB', 0)}")
print(f"Runs with mixed pattern (both equilibria within run): {pattern_counts.get('mixed_within_run', 0)}")

# Show a few examples
print(f"\n=== Sample MC Runs (first 5) ===")
for i in range(min(5, n_mc)):
    mc_data = mc_convergence_pattern[i]
    print(f"MC Run {i+1}: {mc_data['pattern']}")
    print(f"  (A, A): {mc_data['freq_AA']:.3f}, (A, B): {mc_data['freq_AB']:.3f}, "
          f"(B, A): {mc_data['freq_BA']:.3f}, (B, B): {mc_data['freq_BB']:.3f}")

# Conclusion
if pattern_counts.get('mixed_within_run', 0) > n_mc * 0.1:
    print(f"\n=== CONCLUSION ===")
    print("Each MC run randomly selects between the two equilibria within the run.")
    print("The 47%/53% split is achieved by mixing both equilibria within each MC run.")
else:
    print(f"\n=== CONCLUSION ===")
    print("Each MC run converges to one equilibrium (either (A,A) or (B,B)).")
    print("The 47%/53% split is achieved across different MC runs.")
    print(f"  - {pattern_counts.get('converged_to_AA', 0)} runs converged to (A, A)")
    print(f"  - {pattern_counts.get('converged_to_BB', 0)} runs converged to (B, B)")


=== MC Run Convergence Pattern Analysis ===
Total MC runs: 100

Convergence patterns in last 1000 rounds:
  converged_to_AA: 47 runs (47.0%)
  converged_to_BB: 53 runs (53.0%)

=== Detailed Statistics ===
Runs that converged to (A, A): 47
Runs that converged to (B, B): 53
Runs with mixed pattern (both equilibria within run): 0

=== Sample MC Runs (first 5) ===
MC Run 1: converged_to_BB
  (A, A): 0.000, (A, B): 0.000, (B, A): 0.000, (B, B): 1.000
MC Run 2: converged_to_AA
  (A, A): 1.000, (A, B): 0.000, (B, A): 0.000, (B, B): 0.000
MC Run 3: converged_to_BB
  (A, A): 0.000, (A, B): 0.000, (B, A): 0.000, (B, B): 1.000
MC Run 4: converged_to_BB
  (A, A): 0.000, (A, B): 0.000, (B, A): 0.000, (B, B): 1.000
MC Run 5: converged_to_AA
  (A, A): 1.000, (A, B): 0.000, (B, A): 0.000, (B, B): 0.000

=== CONCLUSION ===
Each MC run converges to one equilibrium (either (A,A) or (B,B)).
The 47%/53% split is achieved across different MC runs.
  - 47 runs converged to (A, A)
  - 53 runs converged to (B,

In [4]:
# Parameters
n_rounds = 10000
k = 100
n_mc = 100

### Experiment

In [5]:
v1, v2 = 1.0, 1.0

player1 = (ew.flexible_algorithm, v1, {'k': k, 'h': v1, 'learning_rate': None}) # None = default (sqrt(log(k) / n)), or specify a value like 0.1
player2 = (ew.flexible_algorithm, v2, {'k': k, 'h': v2, 'learning_rate': None})  
results_ew_vs_ew = run_repeated_fpa(player1, player2, n_rounds, n_mc, k=k)
plot_results(results_ew_vs_ew, title="EW vs EW", show_ne_distance=True)

KeyboardInterrupt: 

In [ ]:
v1, v2 = 1.0, 1.0

player1 = (ftl.ftl_algorithm, v1, {'k': k, 'h': v1})
player2 = (ew.flexible_algorithm, v2, {'k': k, 'h': v2, 'learning_rate': None})
results_FTL_vs_ew = run_repeated_fpa(player1, player2, n_rounds, n_mc, k=k)
plot_results(results_FTL_vs_ew, title="FTL vs EW")

MC iteration 10/100 completed
MC iteration 20/100 completed
MC iteration 30/100 completed
MC iteration 40/100 completed
MC iteration 50/100 completed
MC iteration 60/100 completed
MC iteration 70/100 completed
MC iteration 80/100 completed
MC iteration 90/100 completed
MC iteration 100/100 completed
Plot 1 saved to: ../figures/ftl_vs_ew/bid_evolution.png
Plot 2 saved to: ../figures/ftl_vs_ew/regret.png
Plot 4 saved to: ../figures/ftl_vs_ew/utility_distribution.png
Plot 5 saved to: ../figures/ftl_vs_ew/win_rate_distribution.png
Plot 6 saved to: ../figures/ftl_vs_ew/avg_1round_regret_convergence.png

All figures saved to folder: ../figures/ftl_vs_ew


In [ ]:
v1, v2 = 1.0, 1.0

player1 = (uniform_guessing.uniform_guessing_algorithm, v1, {'k': k})
player2 = (ew.flexible_algorithm, v2, {'k': k, 'h': v2, 'learning_rate': None})
results_uniform_vs_ew = run_repeated_fpa(player1, player2, n_rounds, n_mc, k=k)
plot_results(results_uniform_vs_ew, title="Uniform guessing vs EW")

MC iteration 10/100 completed
MC iteration 20/100 completed
MC iteration 30/100 completed
MC iteration 40/100 completed
MC iteration 50/100 completed
MC iteration 60/100 completed
MC iteration 70/100 completed


In [ ]:
v1, v2 = 1.0, 1.0

import numpy as np
optimal_lr = np.sqrt(np.log(k) / n_rounds)
lr_3x = 3 * optimal_lr

player1 = (ew.flexible_algorithm, v1, {'k': k, 'h': v1, 'learning_rate': None})
player2 = (ew.flexible_algorithm, v2, {'k': k, 'h': v2, 'learning_rate': lr_3x})

results_lr_comparison = run_repeated_fpa(player1, player2, n_rounds, n_mc, k=k)
plot_results(results_lr_comparison, title="Optimal LR vs 3x Optimal LR")

MC iteration 10/100 completed
MC iteration 20/100 completed
MC iteration 30/100 completed
MC iteration 40/100 completed
MC iteration 50/100 completed
MC iteration 60/100 completed
MC iteration 70/100 completed
MC iteration 80/100 completed
MC iteration 90/100 completed
MC iteration 100/100 completed
Completed: Optimal LR vs 3x Optimal LR
Plot 1 saved to: ../figures/optimal_lr_vs_3x_optimal_lr_bid_evolution.png
Plot 2 saved to: ../figures/optimal_lr_vs_3x_optimal_lr_regret.png
Plot 4 saved to: ../figures/optimal_lr_vs_3x_optimal_lr_utility_distribution.png
Plot 5 saved to: ../figures/optimal_lr_vs_3x_optimal_lr_win_rate_distribution.png

=== Summary Statistics ===
Player 1:
  Mean Regret: 81.07 ± 5.15
  Mean Utility: 537.02 ± 16.52
  Mean Win Rate: 0.505 ± 0.003

Player 2:
  Mean Regret: 69.00 ± 6.12
  Mean Utility: 532.35 ± 14.66
  Mean Win Rate: 0.495 ± 0.003
Summary statistics saved to: ../data/optimal_lr_vs_3x_optimal_lr_summary.csv
Detailed results saved to: ../data/optimal_lr_vs_3

In [ ]:
v1, v2 = 1.0, 1.0

player1 = (empirical.empirical_algorithm, v1, {'k': k, 'h': v1})
player2 = (ew.flexible_algorithm, v2, {'k': k, 'h': v2, 'learning_rate': None})
results_empirical_vs_ew = run_repeated_fpa(player1, player2, n_rounds, n_mc, k=k)
plot_results(results_empirical_vs_ew, title="Empirical vs EW", show_ne_distance=True)

In [ ]:
from value_generate import generate_value
player1 = (ew.flexible_algorithm, lambda: generate_value('uniform', low=0.0, high=1.0), {'k': k, 'learning_rate': None}) 
player2 = (ew.flexible_algorithm, lambda: generate_value('uniform', low=0.0, high=1.0), {'k': k, 'learning_rate': None}) 
results_ew_vs_ew_from_uniform_distribution = run_repeated_fpa(player1, player2, n_rounds, n_mc, k=k)
plot_results(results_ew_vs_ew_from_uniform_distribution, title="EW vs EW from uniform distribution")

/Users/harashimakoshi/CS332/332Project3/code/algorithm/2_ew.py:147: RuntimeWarning: overflow encountered in power
  powers = (1 + learning_rate) ** (valid_payoffs / h)


Completed: EW vs EW from uniform distribution
Plot 1 saved to: ../figures/ew_vs_ew_from_uniform_distribution_bid_evolution.png
Plot 2 saved to: ../figures/ew_vs_ew_from_uniform_distribution_regret.png
Plot 4 saved to: ../figures/ew_vs_ew_from_uniform_distribution_utility_distribution.png
Plot 5 saved to: ../figures/ew_vs_ew_from_uniform_distribution_win_rate_distribution.png

=== Summary Statistics ===
Player 1:
  Mean Regret: 42.19 ± 0.00
  Mean Utility: 971.94 ± 0.00
  Mean Win Rate: 0.492 ± 0.000

Player 2:
  Mean Regret: 29.05 ± 0.00
  Mean Utility: 967.82 ± 0.00
  Mean Win Rate: 0.508 ± 0.000
Summary statistics saved to: ../data/ew_vs_ew_from_uniform_distribution_summary.csv
Detailed results saved to: ../data/ew_vs_ew_from_uniform_distribution_detailed.csv
Regret history saved to: ../data/ew_vs_ew_from_uniform_distribution_regret_history.csv
Bid history saved to: ../data/ew_vs_ew_from_uniform_distribution_bid_history.csv


## battle of sexes

### Algorithms and Parameters

In [7]:
import numpy as np
import sys, importlib
from pathlib import Path
sys.path.insert(0, str(Path('.').resolve()))
import repeated_bimatrix
from repeated_bimatrix import run_repeated_bimatrix, plot_ne_convergence
from algorithm import bimatrix_wrapper

payoff_matrix = np.array([
    [[2, 4], [0, 0]],  # Player 1 plays A
    [[0, 0], [4, 2]]   # Player 1 plays B
])

print("Payoff Matrix:")
print("        Action A    Action B")
print(f"Action A  ({payoff_matrix[0,0,0]},{payoff_matrix[0,0,1]})      ({payoff_matrix[0,1,0]},{payoff_matrix[0,1,1]})")
print(f"Action B  ({payoff_matrix[1,0,0]},{payoff_matrix[1,0,1]})      ({payoff_matrix[1,1,0]},{payoff_matrix[1,1,1]})")
print("\nPure Nash Equilibria:")
ne_pure = repeated_bimatrix.find_pure_nash_equilibria(payoff_matrix)
for ne in ne_pure:
    print(f"  ({chr(65+ne[0])}, {chr(65+ne[1])})")

Payoff Matrix:
        Action A    Action B
Action A  (2,4)      (0,0)
Action B  (0,0)      (4,2)

Pure Nash Equilibria:
  (A, A)
  (B, B)


In [ ]:
# Parameters for Battle of Sexes game
n_rounds = 10000
n_mc = 100

### Experiments

In [8]:
# Test 1: Flexible vs Flexible
player1_coord = (bimatrix_wrapper.flexible_bimatrix, {'learning_rate': None, 'h': 3.0})
player2_coord = (bimatrix_wrapper.flexible_bimatrix, {'learning_rate': None, 'h': 3.0})

results_flexible_vs_flexible = run_repeated_bimatrix(player1_coord, player2_coord, payoff_matrix, n_rounds, n_mc)
plot_ne_convergence(results_flexible_vs_flexible, title="Flexible vs Flexible (Battle of Sexes)")

MC iteration 10/100 completed
MC iteration 20/100 completed
MC iteration 30/100 completed
MC iteration 40/100 completed
MC iteration 50/100 completed
MC iteration 60/100 completed
MC iteration 70/100 completed
MC iteration 80/100 completed
MC iteration 90/100 completed
MC iteration 100/100 completed
Plot 1 saved to: ../figures/flexible_vs_flexible_battle_of_sexes/flexible_vs_flexible_battle_of_sexes_action_profile_frequencies.png
Plot 2 saved to: ../figures/flexible_vs_flexible_battle_of_sexes/flexible_vs_flexible_battle_of_sexes_final_ne_convergence.png

All figures saved to folder: ../figures/flexible_vs_flexible_battle_of_sexes

=== NE Convergence Summary ===
Pure Nash Equilibria: [(0, 0), (1, 1)]
Final action profile frequencies (last 1000 rounds):
  (A, A): 0.470 ± 0.000 [NE]
  (A, B): 0.000 ± 0.000
  (B, A): 0.000 ± 0.000
  (B, B): 0.530 ± 0.000 [NE]


In [9]:
# Test 2: Flexible vs Empirical (robustness check)
player1_coord2 = (bimatrix_wrapper.flexible_bimatrix, {'learning_rate': None, 'h': 3.0})
player2_coord2 = (bimatrix_wrapper.empirical_bimatrix, {})

results_flexible_vs_empirical = run_repeated_bimatrix(
    player1_coord2, player2_coord2, payoff_matrix, n_rounds, n_mc
)
plot_ne_convergence(results_flexible_vs_empirical, title="Flexible vs Empirical (Battle of Sexes)")

MC iteration 10/100 completed
MC iteration 20/100 completed
MC iteration 30/100 completed
MC iteration 40/100 completed
MC iteration 50/100 completed
MC iteration 60/100 completed
MC iteration 70/100 completed
MC iteration 80/100 completed
MC iteration 90/100 completed
MC iteration 100/100 completed
Plot 1 saved to: ../figures/flexible_vs_empirical_battle_of_sexes/flexible_vs_empirical_battle_of_sexes_action_profile_frequencies.png
Plot 2 saved to: ../figures/flexible_vs_empirical_battle_of_sexes/flexible_vs_empirical_battle_of_sexes_final_ne_convergence.png

All figures saved to folder: ../figures/flexible_vs_empirical_battle_of_sexes

=== NE Convergence Summary ===
Pure Nash Equilibria: [(0, 0), (1, 1)]
Final action profile frequencies (last 1000 rounds):
  (A, A): 0.620 ± 0.000 [NE]
  (A, B): 0.000 ± 0.000
  (B, A): 0.000 ± 0.000
  (B, B): 0.380 ± 0.000 [NE]


In [11]:
# Test 3: FTL vs Flexible (robustness check)
player1_coord3 = (bimatrix_wrapper.ftl_bimatrix, {})
player2_coord3 = (bimatrix_wrapper.flexible_bimatrix, {'learning_rate': None, 'h': 3.0})

results_ftl_vs_flexible = run_repeated_bimatrix(
    player1_coord3, player2_coord3, payoff_matrix, n_rounds, n_mc
)
plot_ne_convergence(results_ftl_vs_flexible, title="FTL vs Flexible (Battle of Sexes)")

MC iteration 10/100 completed
MC iteration 20/100 completed
MC iteration 30/100 completed
MC iteration 40/100 completed
MC iteration 50/100 completed
MC iteration 60/100 completed
MC iteration 70/100 completed
MC iteration 80/100 completed
MC iteration 90/100 completed
MC iteration 100/100 completed
Plot 1 saved to: ../figures/ftl_vs_flexible_battle_of_sexes/ftl_vs_flexible_battle_of_sexes_action_profile_frequencies.png
Plot 2 saved to: ../figures/ftl_vs_flexible_battle_of_sexes/ftl_vs_flexible_battle_of_sexes_final_ne_convergence.png

All figures saved to folder: ../figures/ftl_vs_flexible_battle_of_sexes

=== NE Convergence Summary ===
Pure Nash Equilibria: [(0, 0), (1, 1)]
Final action profile frequencies (last 1000 rounds):
  (A, A): 0.480 ± 0.000 [NE]
  (A, B): 0.000 ± 0.000
  (B, A): 0.000 ± 0.000
  (B, B): 0.520 ± 0.000 [NE]


In [ ]:
# Robustness: Test with different initial conditions
from typing import List, Tuple

def flexible_bimatrix_with_init(player_id: int, round_num: int, history: List[Tuple[int, float, int]],
                                env_state: dict, payoff_matrix, initial_action=None) -> int:
    """Flexible algorithm with optional initial action."""
    if round_num == 0 and initial_action is not None:
        return initial_action
    return bimatrix_wrapper.flexible_bimatrix(player_id, round_num, history, env_state, payoff_matrix)

# Test with different initial conditions

# Case 1: Both start with action A
player1_init_A = (lambda pid, rn, h, es, pm: flexible_bimatrix_with_init(pid, rn, h, es, pm, initial_action=0),
                  {'learning_rate': None, 'h': 3.0})
player2_init_A = (lambda pid, rn, h, es, pm: flexible_bimatrix_with_init(pid, rn, h, es, pm, initial_action=0),
                  {'learning_rate': None, 'h': 3.0})

results_init_AA = run_repeated_bimatrix(
    player1_init_A, player2_init_A, payoff_matrix, n_rounds, n_mc
)
plot_ne_convergence(results_init_AA, title="Flexible vs Flexible (Initial: A, A)")

# Case 2: Both start with action B
player1_init_B = (lambda pid, rn, h, es, pm: flexible_bimatrix_with_init(pid, rn, h, es, pm, initial_action=1),
                  {'learning_rate': None, 'h': 3.0})
player2_init_B = (lambda pid, rn, h, es, pm: flexible_bimatrix_with_init(pid, rn, h, es, pm, initial_action=1),
                  {'learning_rate': None, 'h': 3.0})

results_init_BB = run_repeated_bimatrix(
    player1_init_B, player2_init_B, payoff_matrix, n_rounds, n_mc
)
plot_ne_convergence(results_init_BB, title="Flexible vs Flexible (Initial: B, B)")

# Part 2
- We implement exploitation strategy for repeated FPA
- Exploitation strategy vs 2_ew (Exponential Weight algorithm)

### Algorithms

In [ ]:
import numpy as np
import sys, importlib
from pathlib import Path
sys.path.insert(0, str(Path('algorithm').resolve()))

empirical, ew, exploitation = [importlib.import_module(m) for m in ['1_Empirical', '2_ew', '5_exploitation']]
ftl = importlib.import_module('3_FTL')
uniform_guessing = importlib.import_module('4_uniform_guessing')

import repeated_FPA
from repeated_FPA import run_repeated_fpa, plot_results

### Experiments

In [ ]:
# Parameters
n_rounds = 10000
k = 100
n_mc = 100

In [15]:
v1, v2 = 0.9, 0.3
player1 = (ew.flexible_algorithm, v1, {'k': k, 'h': v1, 'learning_rate': None})  
player2 = (exploitation.exploitation_algorithm, v2, {'k': k, 'h': v2, 'observation_rounds': 5})
results_ew_vs_exploitation = run_repeated_fpa(player1, player2, n_rounds, n_mc, k=k)
print("Completed: EW vs Exploitation")
plot_results(results_ew_vs_exploitation, title="EW vs Exploitation")

MC iteration 10/100 completed
MC iteration 20/100 completed
MC iteration 30/100 completed
MC iteration 40/100 completed
MC iteration 50/100 completed
MC iteration 60/100 completed
MC iteration 70/100 completed
MC iteration 80/100 completed
MC iteration 90/100 completed
MC iteration 100/100 completed
Completed: EW vs Exploitation
Plot 1 saved to: ../figures/ew_vs_exploitation_bid_evolution.png
Plot 2 saved to: ../figures/ew_vs_exploitation_regret.png
Plot 4 saved to: ../figures/ew_vs_exploitation_utility_distribution.png
Plot 5 saved to: ../figures/ew_vs_exploitation_win_rate_distribution.png

=== Summary Statistics ===
Player 1:
  Mean Regret: 61.41 ± 16.64
  Mean Utility: 6496.99 ± 18.73
  Mean Win Rate: 0.903 ± 0.003

Player 2:
  Mean Regret: 401.85 ± 4.30
  Mean Utility: 108.30 ± 2.48
  Mean Win Rate: 0.097 ± 0.003
Summary statistics saved to: ../data/ew_vs_exploitation_summary.csv
Detailed results saved to: ../data/ew_vs_exploitation_detailed.csv
Regret history saved to: ../data/ew

In [ ]:
# Parameters for detail
n_rounds = 10000
k = 100
n_mc = 1

In [17]:
v1, v2 = 0.9, 0.3
# Use default learning rate (sqrt(log(k) / n)) - no need to specify explicitly
player1 = (ew.flexible_algorithm, v1, {'k': k, 'h': v1, 'learning_rate': None})  
player2 = (exploitation.exploitation_algorithm, v2, {'k': k, 'h': v2, 'observation_rounds': 5})
results_ew_vs_exploitation_for_detail = run_repeated_fpa(player1, player2, n_rounds, n_mc, k=k)
print("Completed: EW vs Exploitation")
plot_results(results_ew_vs_exploitation_for_detail, title="EW vs Exploitation (for detail)")

Completed: EW vs Exploitation
Plot 1 saved to: ../figures/ew_vs_exploitation_for_detail_bid_evolution.png
Plot 2 saved to: ../figures/ew_vs_exploitation_for_detail_regret.png
Plot 4 saved to: ../figures/ew_vs_exploitation_for_detail_utility_distribution.png
Plot 5 saved to: ../figures/ew_vs_exploitation_for_detail_win_rate_distribution.png

=== Summary Statistics ===
Player 1:
  Mean Regret: 67.21 ± 0.00
  Mean Utility: 6478.25 ± 0.00
  Mean Win Rate: 0.901 ± 0.000

Player 2:
  Mean Regret: 394.42 ± 0.00
  Mean Utility: 111.49 ± 0.00
  Mean Win Rate: 0.099 ± 0.000
Summary statistics saved to: ../data/ew_vs_exploitation_for_detail_summary.csv
Detailed results saved to: ../data/ew_vs_exploitation_for_detail_detailed.csv
Regret history saved to: ../data/ew_vs_exploitation_for_detail_regret_history.csv
Bid history saved to: ../data/ew_vs_exploitation_for_detail_bid_history.csv
